In [ ]:
from skimage import io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from scipy import stats

%matplotlib inline

path1 = '/home/gs/DataScientist/planet'
trainPath = '/train-tif'
testPath = '/test-tif'

VERBOSE_INTERVAL = 5000

NUM_BINS = 64
MAX_PIX_VAL = 65535


In [ ]:
# definitions

def getImageData (filePath):
    stats = []
    try:
        img = io.imread(filePath)
        r, g, b, nir = img[:, :, 0], img[:, :, 1], img[:, :, 2], img[:, :, 3]
        n = (nir-r) / (nir+r)
        hr, bins = np.histogram(r,NUM_BINS,[0, MAX_PIX_VAL])
        hg, bins = np.histogram(g,NUM_BINS,[0, MAX_PIX_VAL])
        hb, bins = np.histogram(b,NUM_BINS,[0, MAX_PIX_VAL])
        hnir, bins = np.histogram(nir,NUM_BINS,[0, MAX_PIX_VAL])
        ndvi, bins = np.histogram(nir,NUM_BINS,[-1, +1])

        # stats
        stats.append( np.mean(r) )
        stats.append( np.mean(g) )
        stats.append( np.mean(b) )
        stats.append( np.mean(nir) )
        stats.append( np.std(r) )
        stats.append( np.std(g) )
        stats.append( np.std(b) )
        stats.append( np.std(nir) )
        stats.append( np.min(r) )
        stats.append( np.min(g) )
        stats.append( np.min(b) )
        stats.append( np.min(nir) )
        stats.append( np.max(r) )
        stats.append( np.max(g) )
        stats.append( np.max(b) )
        stats.append( np.max(nir) )

        stats.append( scipy.stats.kurtosis(r.ravel()) )
        stats.append( scipy.stats.kurtosis(g.ravel()) )
        stats.append( scipy.stats.kurtosis(b.ravel()) )
        stats.append( scipy.stats.kurtosis(nir.ravel()) )
        stats.append( scipy.stats.skew(r.ravel()) )
        stats.append( scipy.stats.skew(g.ravel()) )
        stats.append( scipy.stats.skew(b.ravel()) )
        stats.append( scipy.stats.skew(nir.ravel()) )
        
        # to array
        stats = np.array(stats)
        
    except:
        print ('  error reading file {}'.format(filePath))
        hr = np.zeros(NUM_BINS)
        hg = np.zeros(NUM_BINS)
        hb = np.zeros(NUM_BINS)
        hnir = np.zeros(NUM_BINS)
        ndvi = np.zeros(NUM_BINS)
        stats = np.zeros(24)

    r = np.concatenate((hr, hg, hb, hnir, ndvi, stats), axis = 0)
    return r

def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=2000):
    br = 0
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.05
    param['max_depth'] = 8
    param['silent'] = 1
    param['eval_metric'] = "logloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.8
    param['colsample_bytree'] = 0.8
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20, verbose_eval = 50)
        br = model.best_iteration
        #print ('best iteration for DICT: {}'.format(br))
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds, verbose_eval = 50)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model, br



In [ ]:
# read Y_train

print ('read train y...')

try:
    Y_train = pd.read_csv(path1+'/train.csv')
except:
    path1 = '/home/ec2-user/DataScientist/planet'
    Y_train = pd.read_csv(path1+'/train.csv')

print ('Y_train lines read: {}'.format(len(Y_train)))

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in Y_train['tags'].values])))
label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}
#print(label_map)
#print
#print(inv_label_map)

Y_trainDict = {}
for i, row in Y_train.iterrows():
    name = row['image_name']
    tags = row['tags']
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    Y_trainDict[name] = targets

#print (Y_trainDict['train_0'])
#print (Y_trainDict['train_1'])
#print (Y_trainDict['train_2'])


# read train images

X_train = [] # arrays

Y_trainAll = []
X_train_id = []
lines = 0

print('Read train images')
path = os.path.join(path1+trainPath, '*.tif')
print (path)
files = glob.glob(path)
for fl in files:
    lines += 1
    if lines % VERBOSE_INTERVAL == 0:
        print ('  files read: {}'.format(lines))
    flbase = os.path.basename(fl)
    r = getImageData(fl)
    X_train.append(r)
    name = flbase.replace('.tif', '')
    X_train_id.append(name)
    Y_trainAll.append(Y_trainDict[name])


Y_trainAll = pd.DataFrame(Y_trainAll)

X_train = pd.DataFrame(X_train)
print ('X_train shape   : {}'.format(X_train.shape))
print ('Y_trainAll shape: {}'.format(Y_trainAll.shape))


X_test = [] # arrays
X_test_id = []
lines = 0 

print('Read test images')
path = os.path.join(path1+testPath, '*.tif')
print (path)
files = glob.glob(path)
for fl in files:
    lines += 1
    if lines % VERBOSE_INTERVAL == 0:
        print ('  files read: {}'.format(lines))
    flbase = os.path.basename(fl)
    r = getImageData(fl)
    X_test.append(r)
    name = flbase.replace('.tif', '')
    X_test_id.append(name)

X_test = pd.DataFrame(X_test)
print (X_test.shape)



In [ ]:

# save data frames of histograms for later use


In [ ]:

# xgb cross validation

# save number of rounds!
brDict = {}

for i in range(0,17):
    print ('target: {} {}'.format(i, inv_label_map[i]))
    Y_train = Y_trainAll.ix[:,i]

    kf = model_selection.KFold(n_splits=3, shuffle=True, random_state=2016)
    for dev_index, val_index in kf.split(range(X_train.shape[0])):
        dev_X, val_X = X_train.iloc[dev_index], X_train.iloc[val_index]
        dev_y, val_y = Y_train.iloc[dev_index], Y_train.iloc[val_index]
        preds, model, br = runXGB(dev_X, dev_y, val_X, val_y)
        brDict[i] = br

print ('bestRounds:')
print (brDict)

In [ ]:
# xgboost predict




predsDF = pd.DataFrame()
for i in range(0,17):
    print ('predicting feature ' + str(i))
    Y_train = Y_trainAll.ix[:,i]
    #print (Y_train.shape)
    preds, model, br = runXGB(X_train, Y_train, X_test, num_rounds=int(brDict[i]*1.33))
    predsDF[i] = preds

print (predsDF.shape)

In [ ]:
print (predsDF.head())
print (X_train_id[0:5])

In [ ]:
# predsDF to prediction file

def mapf (arr):
    res = ''
    for i in range(0,17):
        if arr[i] > 0.5:
            res += inv_label_map[i] + ' '
    res = res.rstrip()
    return res

print(predsDF.shape)

textResults = []

for i, row in predsDF.iterrows():
    #print (i)
    #print (list(row))
    textResults.append ( mapf( list (row)))
    
print (textResults[0:5])

res = pd.DataFrame()
res['image_name'] = X_test_id
res['tags'] = textResults

print (res.head())

res.to_csv('SUB_23.csv', index=False)


